In [1]:
import pandas as pd

In [2]:
file = "data/access-export/tblTexts.xlsx"

In [3]:
df = pd.read_excel(file)

In [4]:
df

,Number,Text,Date,Genre,Corpus,Lower,MeanDate,Upper,Size,Dialect
0,57,abott-e1-p1,c1530,LETTERS_PRIV,PPCEME,1505,1530,1555,430,NaN
1,58,alhatton2-e3-p1,1687,LETTERS_PRIV,PPCEME,1687,1687,1687,1026,NaN
2,59,anhatton-e3-p1,1690-1695,LETTERS_PRIV,PPCEME,1690,1693,1695,820,NaN
3,60,aplumpt-e1-p1,1502-1504,LETTERS_PRIV,PPCEME,1502,1503,1504,378,NaN
4,61,armin-e2-p1,1608,FICTION,PPCEME,1608,1608,1608,6304,NaN
5,62,asch-e1-p1,1563-1568,EDUC_TREATISE,PPCEME,1563,1566,1568,6277,NaN
6,63,aungier-e3-p1,1673-1675,LETTERS_NON-PRIV,PPCEME,1673,1674,1675,1106,NaN
7,64,authnew-e2-p1,1611,BIBLE,PPCEME,1611,1611,1611,14969,NaN
8,65,authold-e2-p1,1611,BIBLE,PPCEME,1611,1611,1611,12972,NaN
9,66,bacon-e2-p1,1605,EDUC_TREATISE,PPCEME,1605,1605,1605,6577,NaN


In [5]:
# create necessary concept-schemes
ecce_genre,_ = SkosConceptScheme.objects.get_or_create(dc_title="ecce-genre")
ecce_dialect,_ = SkosConceptScheme.objects.get_or_create(dc_title="ecce-dialect")

In [6]:
for i, row in df.iterrows():
    temp_genre,_ = SkosConcept.objects.get_or_create(pref_label=row['Genre'])
    temp_genre.save()
    temp_genre.scheme = [ecce_genre]
    temp_dialect,_ = SkosConcept.objects.get_or_create(pref_label=row['Dialect'])
    temp_dialect.save()
    temp_dialect.scheme = [ecce_dialect]

In [7]:
for i, row in df.iterrows():
    temp_corpus,_ = Corpus.objects.get_or_create(name=row['Corpus'])
    temp_corpus.save()
    temp_date,_ = Date.objects.get_or_create(dates=row['MeanDate'])
    temp_date.save()
    temp_text,_ = Text.objects.get_or_create(
        text=row['Text'],
        date=row['Date'],
        genre=SkosConcept.objects.get(pref_label=row['Genre']),
        corpus = temp_corpus,
        mean_date = temp_date,
        lower=int(row['Lower']),
        size = float(row['Size']),
        dialect=SkosConcept.objects.get(pref_label=row['Dialect'])
    )
    temp_text.save()

In [11]:
#for x in SkosConcept.objects.all():
 #   x.delete()

In [13]:
#for x in Text.objects.all():
 #   x.delete()